# global annotations file download wrangle~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
- downloaded from: 
    - GRCh38_latest_genomic.gff.gz : 
        - https://www.ncbi.nlm.nih.gov/genome/guide/human/
        - "RefSeq Reference Genome Annotation" "GRCh38"
    - gencode.v39.annotation.gff3.gz : 
        - https://www.gencodegenes.org/human/
        - "Comprehensive gene annotation" "Gff3" 
- download date: 
    - check the wget output 
        - wget_output.txt = ncbi 
        - wget_output1.txt = gencode 
- genome build 
    - both GRCh38 --> converto to hg18. 
- file format & head: 


In [1]:
!mkdir ../../../../data/global/track_data/annotation/

In [2]:
%%bash
cd ../../../../
wget https://ftp.ncbi.nlm.nih.gov/refseq/H_sapiens/annotation/GRCh38_latest/refseq_identifiers/GRCh38_latest_genomic.gff.gz -P data/global/track_data/annotation/ -o data/global/track_data/annotation/wget_ouput.txt

In [21]:
%%bash 
cd ../../../../
zcat data/global/track_data/annotation/GRCh38_latest_genomic.gff.gz | grep -v "#" | awk '{print $3}' | sort | uniq -c > data/global/track_data/annotation/GRCh38_latest_genomic_summary.txt

In [17]:
%%bash 
cd ../../../../
wget https://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_39/gencode.v39.annotation.gff3.gz -P data/global/track_data/annotation/ -o data/global/track_data/annotation/wget_ouput1.txt

In [22]:
%%bash 
cd ../../../../
zcat data/global/track_data/annotation/gencode.v39.annotation.gff3.gz | grep -v "#" | awk '{print $3}' | sort | uniq -c > data/global/track_data/annotation/gencode.v39.annotation_summary.txt

## **converting the ncbi refseq into chr#**

In [33]:
!wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/database/chromAlias.txt.gz  -P ../../../../data/global/track_data/annotation/

--2022-03-28 16:33:59--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/database/chromAlias.txt.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12065 (12K) [application/x-gzip]
Saving to: ‘../../../../data/global/track_data/annotation/chromAlias.txt.gz’

100%[======================================>] 12,065      --.-K/s   in 0s      

2022-03-28 16:34:00 (84.5 MB/s) - ‘../../../../data/global/track_data/annotation/chromAlias.txt.gz’ saved [12065/12065]



In [1]:
import gzip
with gzip.open("../../../../data/global/track_data/annotation/GRCh38_latest_genomic.gff.gz", "rt") as handle:
        ncbi_annotation_lines = handle.readlines()
with gzip.open("../../../../data/global/track_data/annotation/chromAlias.txt.gz", "rt") as handle:
        conversion_lines = handle.readlines()

In [2]:
conversion_dict = {}
for line in conversion_lines: 
    if "refseq" in line:
        conversion_dict[line.split()[0]] = line.split()[1]
del(conversion_lines)

In [3]:
assert len(set(list(conversion_dict.values()))) == len(conversion_dict.values())# no duplicates
assert len(set(list(conversion_dict.keys()))) == len(conversion_dict.keys())#no duplicates 

In [4]:
from datetime import datetime
timestamp = datetime.now().strftime("%Y/%m/%d").replace("/", "_").replace(":", "_")
with open('../../../../data/global/track_data/annotation/GRCh38_latest_genomic_converted.gff'.format(t=timestamp), 'w') as f: 
    for i,line in enumerate(ncbi_annotation_lines): 
        if line[0] != "#" : # i already double checked that this doesnt leave out some annotation lines
            line_split = line.split("\t")
            refseq,source,feature,start,end,value_1,strand,value_2,ID = line_split
            chr_n = conversion_dict[refseq]
            string_to_write = chr_n+"\t"+source+"\t"+feature+"\t"+start+"\t"+end+"\t"+value_1+"\t"+strand+"\t"+value_2+"\t"+refseq+"\t"+ID
            string_to_write = string_to_write.replace(" ","_")
            f.write(string_to_write)
            
            
            
        

# **combine**

In [7]:
! cat ../../../../data/global/track_data/annotation/GRCh38_latest_genomic_converted.gff > ../../../../data/global/track_data/annotation/global_annotation.gff


In [11]:
%%bash 
cd ../../../../
zcat data/global/track_data/annotation/gencode.v39.annotation.gff3.gz | grep -v "#" >> data/global/track_data/annotation/global_annotation.gff

In [13]:
%%bash 
cd ../../../../
sort -k1,1 -k4,4n data/global/track_data/annotation/global_annotation.gff >  data/global/track_data/annotation/global_annotation_sorted.gff

In [14]:
%%bash 
cd ../../../../
bgzip data/global/track_data/annotation/global_annotation_sorted.gff

In [75]:
%%bash 
cd ../../../../
zgrep "100651914" data/global/track_data/annotation/global_annotation_sorted.gff.gz

NC_000001.11	Curated Genomic	exon	100651914	100652317	.	+	.	ID=id-RPL36AP12;Parent=gene-RPL36AP12;Dbxref=GeneID:100130034,HGNC:HGNC:35897;gbkey=exon;gene=RPL36AP12;pseudo=true
NC_000001.11	Curated Genomic	pseudogene	100651914	100652317	.	+	.	ID=gene-RPL36AP12;Dbxref=GeneID:100130034,HGNC:HGNC:35897;Name=RPL36AP12;description=ribosomal protein L36a pseudogene 12;gbkey=Gene;gene=RPL36AP12;gene_biotype=pseudogene;gene_synonym=RPL36A_2_79;pseudo=true


In [15]:
%%bash 
cd ../../../../
tabix -p gff data/global/track_data/annotation/global_annotation_sorted.gff.gz

In [19]:
%%bash 
cd ../../../../
tabix  data/global/track_data/annotation/global_annotation_sorted.gff.gz chr1:1000-10000

chr1	RefSeq	region	1	248956422	.	+	.	NC_000001.11	ID=NC_000001.11:1..248956422;Dbxref=taxon:9606;Name=1;chromosome=1;gbkey=Src;genome=chromosome;mol_type=genomic_DNA
